In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
pip install nvidia-ml-py3

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=7ac3e576910e1e1ad51830ac90cef6b19fbe2c0ddc82ef2029f80f9784071ac0
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3


In [ ]:
pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00


In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.2 MB/s eta 0:00:00


In [ ]:
pip install transformers[torch]

In [ ]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [ ]:
print_gpu_utilization()

GPU memory occupied: 233 MB.


In [ ]:

answers = pd.read_csv("drive/MyDrive/Levy/english_texts.csv")

In [ ]:
answers

,Unnamed: 0,TEXT,TEXT_NL,cEXT,cNEU,cAGR,cCON,cOPN
0,0,"Well, right now I just woke up from a mid-day ...","Nou, op dit moment werd ik net wakker na een m...",0,1,1,0,1
1,1,"Well, here we go with the stream of consciousn...","Nou, hier gaan we met de stroom van bewustzijn...",0,0,1,0,0
2,2,An open keyboard and buttons to push. The thin...,Een open toetsenbord en knoppen om in te drukk...,0,1,0,1,1
3,3,I can't believe it! It's really happening! M...,Ik kan het niet geloven! Het gebeurt echt! Mij...,1,0,1,1,0
4,4,"Well, here I go with the good old stream of co...","Welnu, hier ga ik weer met de goede oude stroo...",1,0,1,0,1
...,...,...,...,...,...,...,...,...
2958,2958,I am motivated on a day to day basis by the ne...,Ik word dagelijks gemotiveerd door de noodzaak...,1,0,0,1,1
2959,2959,My son is the biggest part of my life and with...,Mijn zoon is het grootste deel van mijn leven ...,1,1,0,0,0
2960,2960,My kids and grandkids are what keeps me motiva...,Mijn kinderen en kleinkinderen houden me elke ...,1,0,1,1,0
2961,2961,My biggest drive is to earn money so I can ret...,Mijn grootste drijfveer is om geld te verdiene...,0,0,0,0,0


In [ ]:
# For cNEU
df_neu = answers[['TEXT', 'cNEU']].rename(columns={'TEXT': 'text', 'cNEU': 'labels'})
df_neu = df_neu[['text', 'labels']]

# For cAGR
df_agr = answers[['TEXT', 'cAGR']].rename(columns={'TEXT': 'text', 'cAGR': 'labels'})
df_agr = df_agr[['text', 'labels']]

# For cCON
df_con = answers[['TEXT', 'cCON']].rename(columns={'TEXT': 'text', 'cCON': 'labels'})
df_con = df_con[['text', 'labels']]

# For cOPN
df_opn = answers[['TEXT', 'cOPN']].rename(columns={'TEXT': 'text', 'cOPN': 'labels'})
df_opn = df_opn[['text', 'labels']]


In [ ]:
df_neu['labels'].value_counts()

1    1482
0    1481
Name: labels, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming df_ext is your DataFrame with 'text' and 'labels' columns

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df_opn['text'], df_opn['labels'], random_state=42, test_size=0.2
)

# Creating a list of dictionaries for the test set
test_data = [{'label': int(label), 'text': text} for label, text in zip(y_test, X_test)]
train_data = [{'label': int(label), 'text': text} for label, text in zip(y_train, X_train)]

# Creating the final data structure
openness = {
    'train': train_data,
    'test': test_data
}


In [ ]:
len(openness['train'][0]['text'])

3927

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=True)

In [ ]:
import torch

def tokenize_and_chunk(texts, labels, tokenizer, chunk_size=510):
    input_id_chunks = []
    mask_chunks = []
    chunked_labels = []

    for text, label in zip(texts, labels):
        # Tokenize without special tokens
        tokens = tokenizer.encode_plus(text, add_special_tokens=False,
                                       return_tensors='pt', truncation=False)

        # Split into chunks of `chunk_size`
        input_ids = tokens['input_ids'][0]
        attention_mask = tokens['attention_mask'][0]
        for i in range(0, len(input_ids), chunk_size):
            input_ids_chunk = input_ids[i:i+chunk_size]
            attention_mask_chunk = attention_mask[i:i+chunk_size]

            # Add [CLS] and [SEP] tokens
            input_ids_chunk = torch.tensor([101] + input_ids_chunk.tolist() + [102])
            attention_mask_chunk = torch.tensor([1] + attention_mask_chunk.tolist() + [1])

            # Pad the sequences
            padding_length = 512 - len(input_ids_chunk)
            input_ids_chunk = torch.cat([input_ids_chunk, torch.zeros(padding_length, dtype=torch.long)])
            attention_mask_chunk = torch.cat([attention_mask_chunk, torch.zeros(padding_length, dtype=torch.long)])

            # Store the chunks
            input_id_chunks.append(input_ids_chunk)
            mask_chunks.append(attention_mask_chunk)
            chunked_labels.append(label)

    # Convert lists to tensors
    input_ids_tensor = torch.stack(input_id_chunks)
    attention_mask_tensor = torch.stack(mask_chunks)

    return input_ids_tensor, attention_mask_tensor, chunked_labels

# Training data
train_texts = [item['text'] for item in openness['train']]
train_labels = [item['label'] for item in openness['train']]
input_ids_tensor_train, attention_mask_tensor_train, chunked_train_labels = tokenize_and_chunk(train_texts, train_labels, tokenizer)

# Preparing the training dictionary
input_dict_train = {
    'input_ids': input_ids_tensor_train.long(),
    'attention_mask': attention_mask_tensor_train.int()
}

# Testing data
test_texts = [item['text'] for item in openness['test']]
test_labels = [item['label'] for item in openness['test']]
input_ids_tensor_test, attention_mask_tensor_test, chunked_test_labels = tokenize_and_chunk(test_texts, test_labels, tokenizer)

# Preparing the testing dictionary
input_dict_test = {
    'input_ids': input_ids_tensor_test.long(),
    'attention_mask': attention_mask_tensor_test.int()
}


In [ ]:
import pandas as pd
from datasets import Dataset

# Convert tensors to lists (for DataFrame compatibility)
input_ids_list_train = input_ids_tensor_train.numpy().tolist()
attention_mask_list_train = attention_mask_tensor_train.numpy().tolist()

input_ids_list_test = input_ids_tensor_test.numpy().tolist()
attention_mask_list_test = attention_mask_tensor_test.numpy().tolist()

# Create DataFrame
df_train = pd.DataFrame({
    'input_ids': input_ids_list_train,
    'attention_mask': attention_mask_list_train,
    'labels': chunked_train_labels
})

df_test = pd.DataFrame({
    'input_ids': input_ids_list_test,
    'attention_mask': attention_mask_list_test,
    'labels': chunked_test_labels
})

# Create Hugging Face Dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import numpy as np
import evaluate

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    f1score = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1score": f1score}

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import BertConfig

# Initializing a RobertaConfig

configuration = BertConfig.from_pretrained(
    "bert-base-cased",
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

In [ ]:
configuration

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased",
    config=configuration
)


In [ ]:
output_dir = '/content/drive/MyDrive/Levy/openness'


In [ ]:
from transformers import TrainingArguments, Trainer, logging

logging.set_verbosity_error()

In [ ]:
print_gpu_utilization()

GPU memory occupied: 9489 MB.


In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=3e-6,
    num_train_epochs=10,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_steps = 4,
    disable_tqdm=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    # gradient_accumulation_steps=8,
    # gradient_checkpointing=True,
    # fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

result = trainer.train()
print_summary(result)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1score
1,0.603600,0.644206,0.649260,0.687943,0.517794,0.590863
2,0.595000,0.646353,0.635335,0.627451,0.626335,0.626892
3,0.564100,0.646705,0.649260,0.692494,0.508897,0.586667
4,0.587200,0.651173,0.645779,0.666667,0.551601,0.603700
5,0.531000,0.659266,0.643168,0.653846,0.574733,0.611742
6,0.521100,0.671097,0.616188,0.608229,0.604982,0.606601
7,0.479600,0.675067,0.634465,0.634470,0.596085,0.614679
8,0.511700,0.679485,0.625762,0.627413,0.578292,0.601852
9,0.485900,0.688135,0.616188,0.609403,0.599644,0.604484
10,0.515900,0.687384,0.619669,0.615955,0.590747,0.603088


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Time: 849.41
Samples/second: 56.16
GPU memory occupied: 9489 MB.


In [ ]:
# trainer.evaluate()

In [ ]:
# print_gpu_utilization()


Hyperparamater tuning

In [ ]:
# output_dir_hyper = '/content/drive/MyDrive/Levy/hyper_param_ext'


In [ ]:
# %pip install wandb

In [ ]:
# def wandb_hp_space(trial):
#     return {
#         "method": "bayes",
#         "metric": {"name": "objective", "goal": "minimize"},
#         "parameters": {
#             "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 5e-5},
#             "per_device_train_batch_size": {"values": [4, 8]},
#             "num_train_epochs": {"values": [5, 10]},
#         },
#     }

In [ ]:
# def model_init():
#     return BertForSequenceClassification.from_pretrained(
#     "bert-base-cased", num_labels=2, id2label=id2label, label2id=label2id
# )

In [ ]:
# training_args_hyper = TrainingArguments(
#     output_dir=output_dir_hyper,
#     per_device_eval_batch_size=16,
#     per_device_train_batch_size=16,
#     weight_decay=0.01,
#     logging_steps = 4,
#     disable_tqdm=False,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     push_to_hub=False,
#     gradient_accumulation_steps=8,
#     gradient_checkpointing=True,
#     fp16=True,
# )

In [ ]:
# trainer = Trainer(
#     model=None,
#     model_init=model_init,
#     args=training_args_hyper,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     data_collator=data_collator,
# )

In [ ]:
# best_trial = trainer.hyperparameter_search(
#     direction="minimize",
#     backend="wandb",
#     hp_space=wandb_hp_space,
#     n_trials=15,
# )